# Яндекс.Музыка

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

**Цель исследования** — проверьте три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей вы получите из файла `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составьте первое представление о данных Яндекс.Музыки.




Импортируем используемые в проекте библиотеки.

In [1]:
import pandas as pd
from datetime import datetime
from json import dumps

Прочитайте файл `yandex_music_project.csv` из папки `/datasets` и сохраните его в переменной `df`:

In [2]:
try:
    df = pd.read_csv('/datasets/yandex_music_project.csv')
except:
    df = pd.read_csv('datasets/yandex_music_project.csv')

In [3]:
# df.to_csv('yandex_music_project.csv', index=False)

Выводим
на экран первые десять строк таблицы:

In [4]:
display(df.head(10))

userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            Moscow  08:34:34     Monday  
5  Saint-Petersburg  13:09:41     Friday  
6            Moscow  13:00:07  Wednesday  
7            Moscow  20:47:49  Wednesday  
8            Moscow  09:17:40     Friday  
9  Saint-Petersburg  21:20:49  Wednesday

Получаем общую информацию о таблице:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

В названиях колонок видны три нарушения стиля:
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Не хватает snake_case, должно быть user_id.



Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных
Исправьте стиль в заголовках столбцов, исключите пропуски. Затем проверьте данные на дубликаты.

### Стиль заголовков
Выведите на экран названия столбцов:

In [6]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Приведите названия в соответствие с хорошим стилем:
* несколько слов в названии запишите в «змеином_регистре»,
* все символы сделайте строчными,
* устраните пробелы.

Для этого переименуйте колонки так:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [7]:
# Используем функцию map() с str.lower чтобы сделать все строчки нижнего регистра, а потом map() с lambda функцией для удаления лишних пробелов.
# В данном случае strip() - метод string, а элементы списка как раз строки.
df.columns = df.columns.map(str.lower).map(lambda x: x.strip())
# user_id меняем вручную.
df = df.rename(columns={'userid':'user_id'})

Проверьте результат. Для этого ещё раз выведите на экран названия столбцов:

In [8]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений
Сначала посчитайте, сколько в таблице пропущенных значений. Для этого достаточно двух методов `pandas`:

In [9]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски не важны для вашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить и эти пропуски явными обозначениями,
* оценить, насколько они повредят расчётам. 

Замените пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`. Для этого создайте список `columns_to_replace`, переберите его элементы циклом `for` и для каждого столбца выполните замену пропущенных значений:

In [10]:
columns_to_replace = ['track','artist','genre']

for col in columns_to_replace:
    df[col] = df[col].fillna('unknown')

Убедитесь, что в таблице не осталось пропусков. Для этого ещё раз посчитайте пропущенные значения.

In [11]:
# Второй вызов Sum() чтобы убедиться что NaN больше нет.
df.isna().sum().sum()

0

### Дубликаты
Посчитайте явные дубликаты в таблице одной командой:

In [12]:
df.duplicated().sum()

3826

Вызовите специальный метод `pandas`, чтобы удалить явные дубликаты:

In [13]:
df = df.drop_duplicates().reset_index(drop=True)

Ещё раз посчитайте явные дубликаты в таблице — убедитесь, что полностью от них избавились:

In [14]:
df.duplicated().sum()

0

Теперь избавьтесь от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Выведите на экран список уникальных названий жанров, отсортированный в алфавитном порядке. Для этого:
* извлеките нужный столбец датафрейма, 
* примените к нему метод сортировки,
* для отсортированного столбца вызовите метод, который вернёт уникальные значения из столбца.

In [15]:
# Находим уникальные жанры
uniqe_genres = df.genre.sort_values().unique()

# Выводим их в читабельном виде:
i = 0
for n in uniqe_genres:
    i += 1
    print(f'{n:<20}', end='\t')
    if i > 5:
        i = 0; print('')

acid                	acoustic            	action              	adult               	africa              	afrikaans           	
alternative         	alternativepunk     	ambient             	americana           	animated            	anime               	
arabesk             	arabic              	arena               	argentinetango      	art                 	audiobook           	
author              	avantgarde          	axé                 	baile               	balkan              	beats               	
bigroom             	black               	bluegrass           	blues               	bollywood           	bossa               	
brazilian           	breakbeat           	breaks              	broadway            	cantautori          	cantopop            	
canzone             	caribbean           	caucasian           	celtic              	chamber             	chanson             	
children            	chill               	chinese             	choral              	christian           	christ

Просмотрите список и найдите неявные дубликаты названия `hiphop`. Это могут быть названия с ошибками или альтернативные названия того же жанра.

Вы увидите следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

Чтобы очистить от них таблицу, напишите функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

Функция должна исправить колонку `genre` в таблице `df`: заменить каждое значение из списка `wrong_genres` на значение из `correct_genre`.

In [16]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for genre in wrong_genres:
        df['genre'] = df['genre'].replace(genre, correct_genre)

Вызовите `replace_wrong_genres()` и передайте ей такие аргументы, чтобы она устранила неявные дубликаты: вместо `hip`, `hop` и `hip-hop` в таблице должно быть значение `hiphop`:

In [17]:
replace_wrong_genres(['hip', 'hop', 'hip-hop'], 'hiphop')
# Исправиляем ещё пару неявных дубликатов.
replace_wrong_genres(['электроника'], 'electronic')
replace_wrong_genres(['afrikaans'], 'africa')
replace_wrong_genres(['türkçe'], 'türk')

Проверьте, что заменили неправильные названия:

*   hip
*   hop
*   hip-hop

Выведите отсортированный список уникальных значений столбца `genre`:

In [18]:
# Не стал ещё раз выводить список, привел разницу. Так нагляднее.
uniqe_genres_updated = df.genre.sort_values().unique()
print('Уникальных значений стало меньше на:', len(uniqe_genres) - len(uniqe_genres_updated))

Уникальных значений стало меньше на: 6


**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Вы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения вы заменили на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделите пользователей Москвы и Санкт-Петербурга
* Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


Для тренировки сначала выполните каждый из расчётов по отдельности. 

Оцените активность пользователей в каждом городе. Сгруппируйте данные по городу и посчитайте прослушивания в каждой группе.



In [19]:
# Создадим списки дней и городов которые нам интересны
days = ['Monday', 'Wednesday', 'Friday']
cities = list(df.city.sort_values().unique())

# Добавим словари
count_for_city = {}
count_on_day = {}

for c in cities:
    for d in days:
        # Оборачиваем в int() иначе json.dumps выдает ошибку
        count_on_day[d] = int(df[(df['city'] == c) &
                             (df['day'] == d)]['track'].count()
                             )
    # Без hard copy() значения в Мск и Спб совпадут
    count_for_city[c] = count_on_day.copy()

In [20]:
def user_counter(city = None, day = None, sum_by_day = None):
    if city == None and day == None and sum_by_day != True:
        return print(dumps(count_for_city, indent=4))
    
    if city in cities and day == None:
        return print(f'Число прослушиваний: {sum(count_for_city[city].values())}')
    
    if sum_by_day:
        for d in days:
            count = 0
            for c in cities:
                count += count_for_city[c][d]
            print(f'Прослушиваний в {d}: {count}')
    
    if city in cities and day in days:
        # Функция будет реализована ниже по заданию
        return number_tracks(day, city)

In [21]:
# Выводим получившийся результат
user_counter()

{
    "Moscow": {
        "Monday": 15740,
        "Wednesday": 11056,
        "Friday": 15945
    },
    "Saint-Petersburg": {
        "Monday": 5614,
        "Wednesday": 7003,
        "Friday": 5895
    }
}


### Подсчёт прослушиваний в каждом городе

In [22]:
# По Москве:
user_counter('Moscow')

Число прослушиваний: 42741


In [23]:
# По Санкт-Петербургу:
user_counter('Saint-Petersburg')

Число прослушиваний: 18512


В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируйте данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. Учтите, что в данных есть информация только о прослушиваниях только за эти дни.


In [24]:
# Подсчёт прослушиваний в каждый из трёх дней
user_counter(sum_by_day=True)

Прослушиваний в Monday: 21354
Прослушиваний в Wednesday: 18059
Прослушиваний в Friday: 21840


В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Вы видели, как работает группировка по городу и по дням недели. Теперь напишите функцию, которая объединит два эти расчёта.

Создайте функцию `number_tracks()`, которая посчитает прослушивания для заданного дня и города. Ей понадобятся два параметра:
* день недели,
* название города.

В функции сохраните в переменную строки исходной таблицы, у которых значение:
  * в колонке `day` равно параметру `day`,
  * в колонке `city` равно параметру `city`.

Для этого примените последовательную фильтрацию с логической индексацией.

Затем посчитайте значения в столбце `user_id` получившейся таблицы. Результат сохраните в новую переменную. Верните эту переменную из функции.

In [25]:
# <создание функции number_tracks()> с двумя параметрами: day, city.

def number_tracks(day, city):
    # Используем фильтрацию
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    
    # Функция для подсчёта прослушиваний для конкретного города и дня.
    # С помощью последовательной фильтрации с логической индексацией она 
    # сначала получит из исходной таблицы строки с нужным днём,
    # затем из результата отфильтрует строки с нужным городом,
    # методом count() посчитает количество значений в колонке user_id. 
    # Это количество функция вернёт в качестве результата
    return track_list_count

Вызовите `number_tracks()` шесть раз, меняя значение параметров — так, чтобы получить данные для каждого города в каждый из трёх дней.

In [26]:
# количество прослушиваний в Москве по понедельникам
user_counter('Moscow','Monday')

15740

In [27]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
user_counter('Saint-Petersburg','Monday')

5614

In [28]:
# количество прослушиваний в Москве по средам
user_counter('Moscow','Wednesday')

11056

In [29]:
# количество прослушиваний в Санкт-Петербурге по средам
user_counter('Saint-Petersburg','Wednesday')

7003

In [30]:
# количество прослушиваний в Москве по пятницам
user_counter('Moscow','Friday')

15945

In [31]:
# количество прослушиваний в Санкт-Петербурге по пятницам
user_counter('Saint-Petersburg','Friday')

5895

Создайте c помощью конструктора `pd.DataFrame` таблицу, где
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые вы получили с помощью `number_tracks`.

In [32]:
# Таблица с результатами
result_tab = pd.DataFrame(data=[['MSK', 15740, 11056, 15945],
                                ['SPB', 5614, 7003, 5895]],
                          columns=['city', 'monday', 'wednesday', 'friday'])
result_tab

city  monday  wednesday  friday
0  MSK   15740      11056   15945
1  SPB    5614       7003    5895

In [33]:
# Альтернативный вариант
result_tab = pd.DataFrame(count_for_city)
result_tab

Moscow  Saint-Petersburg
Monday      15740              5614
Wednesday   11056              7003
Friday      15945              5895

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:
* по Москве — в `moscow_general`;
* по Санкт-Петербургу — в `spb_general`.

In [34]:
# Переведем строки во время. В качестве альтернативы можно было идти через split(:)..
df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%H:%M:%S').time())

In [35]:
# Получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df.city == 'Moscow']

In [36]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df.city == 'Saint-Petersburg']

Создайте функцию `genre_weekday()` с четырьмя параметрами:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

Функция должна вернуть информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [37]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
def genre_weekday(table, day, time1, time2):
    # Переводим строки к временному типу данных
    time1 = datetime.strptime(time1, '%H:%M').time()
    time2 = datetime.strptime(time2, '%H:%M').time()
    
    # Фильтруем фрейм
    genre_df = table[(table['day'] == day) &
                     (table['time'] > time1) &
                     (table['time'] < time2)]
    # сгруппировать датафрейм genre_df по столбцу genre, взять один из его
    # столбцов и посчитать методом count() количество записей для каждого из
    # присутствующих жанров, получившийся Series записать в переменную
    # genre_df_count
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    # отсортировать genre_df_count по убыванию встречаемости и сохранить
    # в переменную genre_df_sorted
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    return genre_df_sorted.head(10)

Cравните результаты функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [38]:
# вызов функции для утра понедельника в Москве (вместо df — таблица moscow_general)
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [39]:
# вызов функции для утра понедельника в Петербурге (вместо df — таблица spb_general)
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [40]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [41]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

Сгруппируйте таблицу `moscow_general` по жанру и посчитайте прослушивания треков каждого жанра методом `count()`. Затем отсортируйте результат в порядке убывания и сохраните его в таблице `moscow_genres`.

In [42]:
# Одной строкой: группировка таблицы moscow_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в moscow_genres
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведите на экран первые десять строк `moscow_genres`:

In [43]:
# Рросмотр первых 10 строк moscow_genres
moscow_genres.head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Теперь повторите то же и для Петербурга.

Сгруппируйте таблицу `spb_general` по жанру. Посчитайте прослушивания треков каждого жанра. Результат отсортируйте в порядке убывания и сохраните в таблице `spb_genres`:


In [44]:
# Одной строкой: группировка таблицы spb_general по столбцу 'genre', 
# подсчёт числа значений 'genre' в этой группировке методом count(), 
# сортировка получившегося Series в порядке убывания и сохранение в spb_genres
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Выведите на экран первые десять строк `spb_genres`:

In [45]:
# Просмотр первых 10 строк spb_genres
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

<div class="alert alert-info">Дополнительно хотелось разобраться с 'unknown' жанрами. Но из таблицы ниже видно, что возможно проблема возможно связана с пользователями у которых любимый жанр 'unknown'</div>

In [46]:
df_filtered = pd.DataFrame(df.groupby(['user_id','genre'])['track'].count()).reset_index()
df_filtered[df_filtered['track']>1].sort_values('track', ascending=False).head(10)

user_id      genre  track
33927  A8AE9169    unknown     71
10903  414F229D    unknown     48
49432  EED05948    unknown     36
9262   39EDC574    unknown     30
4642   24F3E2DB    unknown     28
15025  53B9A51D  classical     25
42328  CE89F112      blues     23
44111  D6A03F1E      dance     20
21027  6F81439E    unknown     20
37417  B851E1A5        pop     19

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

Вы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из данных одного сервиса не всегда можно сделать вывод о всех жителях города.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных. 
С методами проверок гипотез вы ещё познакомитесь в следующих темах.